In [4]:
from platform import python_version
python_version()

'3.8.10'

In [21]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

assessments = pd.read_csv('data/source_data/assessments.csv')
studentInfo = pd.read_csv('data/source_data/studentInfo.csv')

In [3]:
query_params = {"code_module" : "FFF",
                "code_presentation" : "2014J"}

In [63]:
assessments_hier = assessments.loc[assessments['code_module']==query_params['code_module']].loc[assessments['code_presentation']==query_params['code_presentation']].drop(columns=['code_module', 'code_presentation']).set_index(['id_assessment']).sort_index()
assessments_hier

,assessment_type,date,weight
id_assessment,,,
34899,TMA,24.0,12.5
34900,TMA,52.0,12.5
34901,TMA,94.0,25.0
34902,TMA,136.0,25.0
34903,TMA,199.0,25.0
34904,CMA,241.0,0.0
34905,CMA,241.0,0.0
34906,CMA,241.0,0.0
34907,CMA,241.0,0.0


In [36]:
studentInfo_hier = studentInfo.loc[studentInfo['code_module']==query_params['code_module']].loc[studentInfo['code_presentation']==query_params['code_presentation']].drop(columns=['code_module', 'code_presentation']).set_index(['id_student']).sort_index()
# get only first try students
studentInfo_hier = studentInfo_hier.loc[studentInfo_hier.num_of_prev_attempts==0].drop(columns=['num_of_prev_attempts'])
chosen_id_students = studentInfo_hier.index.values
studentInfo_hier.head()

array([  35747,   35796,   40857, ..., 2686035, 2694139, 2702660])

In [79]:
# fake_fill = pd.DataFrame(data = {col_name: [np.nan for _ in range(assessments_hier.index.values.shape[0])]
#                                  for col_name in ['date_submitted','is_banked','score']}.,
#                          index = )
# fake_fill

{col_name: [np.nan for _ in range(assessments_hier.index.values.shape[0])]
                                 for col_name in ['date_submitted','is_banked','score']}|{"3":3}

# studentAssessment_hier.loc[31296].set_index(['id_assessment']).head().combine_first(fake_fill).sort_index()

TypeError: unsupported operand type(s) for |: 'dict' and 'dict'

In [27]:
def get_right_studentAssessment_data(query_params = {"code_module" : "FFF", "code_presentation" : "2014J"}, chosen_id_students):
    studentAssessment_tmp = pd.read_csv('data/source_data/studentAssessment.csv')
    studentAssessment_tmp = studentAssessment.loc[studentAssessment[['id_assessment']].applymap(lambda x: x in assessments_hier.index.values)['id_assessment'].values].set_index(['id_student', 'id_assessment']).sort_index().reset_index().set_index(['id_student'])
    complete_empty_studentAssessment_tmp = pd.DataFrame(data=)
# studentAssessment_hier.loc[chosen_id_students]


,id_assessment,date_submitted,is_banked,score
id_student,,,,
31296,34899,24,0,78.0
31296,34900,58,0,40.0
31296,34901,97,0,38.0
31296,34902,141,0,43.0
31296,34903,199,0,69.0


In [26]:
studentInfo_hier.num_of_prev_attempts.value_counts()

0    2034
1     244
2      66
3      14
4       6
6       1
Name: num_of_prev_attempts, dtype: int64

In [42]:
# filtering students
# 1) first attempt - to get behaviour

In [55]:
doll1 = pd.DataFrame(data=[[2,3], [np.nan, 4]])
doll2 = pd.DataFrame(data=[[2,3], [5, np.nan], [3,3]], index = [3,1,2])

In [56]:
doll1

,0,1
0,2.0,3
1,NaN,4


In [58]:
doll2

,0,1
3,2,3.0
1,5,NaN
2,3,3.0


In [59]:
doll1.combine_first(doll2)

,0,1
0,2.0,3.0
1,5.0,4.0
2,3.0,3.0
3,2.0,3.0
